<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [1]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [2]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [3]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [4]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [56]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
        print(core)
        if core['core'] != None:
            response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
            Block.append(response['block'])
            ReusedCount.append(response['reuse_count'])
            Serial.append(response['serial'])
        else:
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)
        Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
        Flights.append(core['flight'])
        GridFins.append(core['gridfins'])
        Reused.append(core['reused'])
        Legs.append(core['legs'])
        LandingPad.append(core['landpad'])

Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [6]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [7]:
response = requests.get(spacex_url)

Check the content of the response


In [57]:
#print(response.content)

You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [9]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

We should see that the request was successfull with the 200 status response code


In [10]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [11]:
# Use json_normalize meethod to convert the json result into a dataframe
#response.json()
data = pd.json_normalize(response.json())

Using the dataframe <code>data</code> print the first 5 rows


In [12]:
# Get the head of the dataframe
data["cores"].head()

0    [{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]
1    [{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]
2    [{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]
3    [{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]
4    [{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]
Name: core

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [13]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data['cores'] = data['cores'].map(lambda x : x[0]).map(lambda x: x['core'])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]
data

,rocket,payloads,launchpad,cores,flight_number,date_utc,date
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,5e9e4502f5090995de566f86,5e9e289df35918033d3b2623,1,2006-03-24T22:30:00.000Z,2006-03-24
1,5e9d0d95eda69955f709d1eb,5eb0e4b6b6c3bb0006eeb1e2,5e9e4502f5090995de566f86,5e9e289ef35918416a3b2624,2,2007-03-21T01:10:00.000Z,2007-03-21
3,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e5,5e9e4502f5090995de566f86,5e9e289ef3591855dc3b2626,4,2008-09-28T23:15:00.000Z,2008-09-28
4,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e6,5e9e4502f5090995de566f86,5e9e289ef359184f103b2627,5,2009-07-13T03:35:00.000Z,2009-07-13
5,5e9d0d95eda69973a809d1ec,5eb0e4b7b6c3bb0006eeb1e7,5e9e4501f509094ba4566f84,5e9e289ef359185f2b3b2628,6,2010-06-04T18:45:00.000Z,2010-06-04
...,...,...,...,...,...,...,...
101,5e9d0d95eda69973a809d1ec,5ef6a4600059c33cee4a829e,5e9e4502f509094188566f88,5ef670f10059c33cee4a826c,102,2020-09-03T12:46:00.000Z,2020-09-03
102,5e9d0d95eda69973a809d1ec,5ef6a48e0059c33cee4a829f,5e9e4502f509094188566f88,5e9e28a7f3591817f23b2663,103,2020-10-06T11:29:00.000Z,2020-10-06
103,5e9d0d95eda69973a809d1ec,5ef6a4d50059c33cee4a82a1,5e9e4502f509094188566f88,5e9e28a6f35918c0803b265c,104,2020-10-18T12:25:00.000Z,2020-10-18
104,5e9d0d95eda69973a809d1ec,5ef6a4ea0059c33cee4a82a2,5e9e4501f509094ba4566f84,5ef670f10059c33cee4a826c,105,2020-10-24T15:31:00.000Z,2020-10-24


* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [14]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [15]:
BoosterVersion

[]

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [21]:
# Call getBoosterVersion
def getBoosterVersion(data):
    for rocket_id in data['rocket']:
        if rocket_id:
            try:
                response = requests.get(f"https://api.spacexdata.com/v4/rockets/{rocket_id}")
                response.raise_for_status()
                rocket_data = response.json()
                if 'name' in rocket_data:
                    BoosterVersion.append(rocket_data['name'])
                else:
                    BoosterVersion.append('Unknown')
            except requests.exceptions.RequestException as e:
                print(f"Request failed for rocket ID {rocket_id}: {e}")
                BoosterVersion.append('Unknown')
            except ValueError:
                print(f"Failed to parse JSON response for rocket ID {rocket_id}")
                BoosterVersion.append('Unknown')

the list has now been update 


In [22]:
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

we can apply the rest of the  functions here:


In [23]:
# Call getLaunchSite
#getLaunchSite(data)
def getLaunchSite(data):
    for launchpad_id in data['launchpad']:
        if launchpad_id:
            try:
                response = requests.get(f"https://api.spacexdata.com/v4/launchpads/{launchpad_id}")
                response.raise_for_status()
                launchpad_data = response.json()
                LaunchSite.append(launchpad_data.get('name', 'Unknown'))
                Longitude.append(launchpad_data.get('longitude', 'Unknown'))
                Latitude.append(launchpad_data.get('latitude', 'Unknown'))
            except requests.exceptions.RequestException as e:
                print(f"Request failed for launchpad ID {launchpad_id}: {e}")
                LaunchSite.append('Unknown')
                Longitude.append('Unknown')
                Latitude.append('Unknown')
            except ValueError:
                print(f"Failed to parse JSON response for launchpad ID {launchpad_id}")

In [24]:
# Call getPayloadData
#getPayloadData(data)
def getPayloadData(data):
    for payload_id in data['payloads']:
        if payload_id:
            try:
                response = requests.get(f"https://api.spacexdata.com/v4/payloads/{payload_id}")
                response.raise_for_status()
                payload_data = response.json()
                PayloadMass.append(payload_data.get('mass_kg', 'Unknown'))
                Orbit.append(payload_data.get('orbit', 'Unknown'))
            except requests.exceptions.RequestException as e:
                print(f"Request failed for payload ID {payload_id}: {e}")
                PayloadMass.append('Unknown')
                Orbit.append('Unknown')
            except ValueError:
                print(f"Failed to parse JSON response for payload ID {payload_id}")
                PayloadMass.append('Unknown')
                Orbit.append('Unknown')

In [58]:
# Call getCoreData
#getCoreData(data)
def getCoreData(data):
    for core_id in data['cores']:
        if core_id:
            try:
                response = requests.get(f"https://api.spacexdata.com/v4/cores/{core_id}")
                response.raise_for_status()
                core_data = response.json()
                Outcome.append(core_data.get('landing_success', 'Unknown'))
                Flights.append(core_data.get('flight', 'Unknown'))
                GridFins.append(core_data.get('gridfins', 'Unknown'))
                Reused.append(core_data.get('reused', 'Unknown'))
                Legs.append(core_data.get('legs', 'Unknown'))
                LandingPad.append(core_data.get('landpad', 'Unknown'))
                Block.append(core_data.get('block', 'Unknown'))
                ReusedCount.append(core_data.get('reuse_count', 'Unknown'))
                Serial.append(core_data.get('serial', 'Unknown'))
            except requests.exceptions.RequestException as e:
                print(f"Request failed for core ID {core_id}: {e}")
                Outcome.append('Unknown')
                Flights.append('Unknown')
                GridFins.append('Unknown')
                Reused.append('Unknown')
                Legs.append('Unknown')
                LandingPad.append('Unknown')
                Block.append('Unknown')
                ReusedCount.append('Unknown')
                Serial.append('Unknown')
            except ValueError:
                print(f"Failed to parse JSON response for core ID {core_id}")
                Outcome.append('Unknown')
                Flights.append('Unknown')
                GridFins.append('Unknown')
                Reused.append('Unknown')
                Legs.append('Unknown')
                LandingPad.append('Unknown')
                Block.append('Unknown')
                ReusedCount.append('Unknown')
                Serial.append('Unknown')
                
getCoreData(data)

Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [34]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

print(len(list(data['flight_number'])))
print(len(list(data['date'])))
print(len(BoosterVersion))
print(len())



94
94
94


Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [59]:
# Create a data from launch_dict

#pd.DataFrame.from_dict(launch_dict)
# Create the launch_dict with the collected data
launch_dict = {
    'BoosterVersion': BoosterVersion,
    'PayloadMass': PayloadMass,
    'Orbit': Orbit,
    'LaunchSite': LaunchSite,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    'Reused': Reused,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Block': Block,
    'ReusedCount': ReusedCount,
    'Serial': Serial,
    'Longitude': Longitude,
    'Latitude': Latitude,
    'FlightNumber': data['flight_number'].tolist(),
    'Date': data['date'].tolist()
}


print({k:len(v) for k, v in launch_dict.items()})


# Convert the launch_dict to a DataFrame
launch_data_df = pd.DataFrame(launch_dict)


{'BoosterVersion': 94, 'PayloadMass': 94, 'Orbit': 94, 'LaunchSite': 94, 'Outcome': 94, 'Flights': 94, 'GridFins': 94, 'Reused': 94, 'Legs': 94, 'LandingPad': 94, 'Block': 94, 'ReusedCount': 94, 'Serial': 94, 'Longitude': 94, 'Latitude': 94, 'FlightNumber': 94, 'Date': 94}


Show the summary of the dataframe


In [60]:
# Show the head of the dataframe
launch_data_df.head()

,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,FlightNumber,Date
0,Falcon 1,20.0,LEO,Kwajalein Atoll,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,0,Merlin1A,167.743129,9.047721,1,2006-03-24
1,Falcon 1,NaN,LEO,Kwajalein Atoll,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,0,Merlin2A,167.743129,9.047721,2,2007-03-21
2,Falcon 1,165.0,LEO,Kwajalein Atoll,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,0,Merlin2C,167.743129,9.047721,4,2008-09-28
3,Falcon 1,200.0,LEO,Kwajalein Atoll,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,0,Merlin3C,167.743129,9.047721,5,2009-07-13
4,Falcon 9,NaN,LEO,CCSFS SLC 40,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1.0,0,B0003,-80.577366,28.561857,6,2010-06-04


## Task 2: Filter the dataframe to only include `Falcon 9` launches

Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [63]:
# Hint data['BoosterVersion']!='Falcon 1'
data_falcon9=launch_data_df[launch_data_df["BoosterVersion"] == "Falcon 9"]

Now that we have removed some values we should reset the FlgihtNumber column


In [64]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,FlightNumber,Date
4,Falcon 9,NaN,LEO,CCSFS SLC 40,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1.0,0,B0003,-80.577366,28.561857,1,2010-06-04
5,Falcon 9,525.0,LEO,CCSFS SLC 40,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1.0,0,B0005,-80.577366,28.561857,2,2012-05-22
6,Falcon 9,677.0,ISS,CCSFS SLC 40,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1.0,0,B0007,-80.577366,28.561857,3,2013-03-01
7,Falcon 9,500.0,PO,VAFB SLC 4E,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1.0,0,B1003,-120.610829,34.632093,4,2013-09-29
8,Falcon 9,3170.0,GTO,CCSFS SLC 40,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1.0,0,B1004,-80.577366,28.561857,5,2013-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Falcon 9,15600.0,VLEO,KSC LC 39A,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,5.0,12,B1060,-80.603956,28.608058,86,2020-09-03
90,Falcon 9,15600.0,VLEO,KSC LC 39A,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,5.0,13,B1058,-80.603956,28.608058,87,2020-10-06
91,Falcon 9,15600.0,VLEO,KSC LC 39A,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,5.0,12,B1051,-80.603956,28.608058,88,2020-10-18
92,Falcon 9,15600.0,VLEO,CCSFS SLC 40,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,5.0,12,B1060,-80.577366,28.561857,89,2020-10-24


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [65]:
data_falcon9.isnull().sum()

BoosterVersion    0
PayloadMass       5
Orbit             0
LaunchSite        0
Outcome           0
Flights           0
GridFins          0
Reused            0
Legs              0
LandingPad        0
Block             0
ReusedCount       0
Serial            0
Longitude         0
Latitude          0
FlightNumber      0
Date              0
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [66]:
# Calculate the mean value of PayloadMass column
payload_mass_mean = launch_data_df['PayloadMass'].replace('Unknown', np.nan).astype(float).mean()
print(f"Mean PayloadMass: {payload_mass_mean}")

# Replace the np.nan values with its mean value
launch_data_df['PayloadMass'] = launch_data_df['PayloadMass'].replace('Unknown', np.nan).astype(float)
launch_data_df['PayloadMass'].fillna(payload_mass_mean, inplace=True)

Mean PayloadMass: 5919.16534090909


You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|


Copyright © 2021 IBM Corporation. All rights reserved.
